# Create the article retrieval tool

## Let's start with creating the functions

In [7]:
import os
import openai
import requests
from pydantic import BaseModel, Field
from typing import Optional

import datetime
from langchain.tools.render import format_tool_to_openai_function
from langchain.chat_models import ChatOpenAI
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

%pdb on # for debugging

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

from langchain.agents import tool

lm_name = "gpt-3.5-turbo-0613"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "lawassist"

Incorrect argument. Use on/1, off/0, or nothing for a toggle.


In [8]:
# Define the input schema
class ParagraphFinder(BaseModel):
    articleNum: int = Field(..., description="The article number to search for. Must be specified.")
    articleNumMinor: Optional[str] = Field(None, description="The minor version of the article to search for. E.g. in Art. 123b it would be 'b'")
    paragraphNum: Optional[int] = Field(None, description="The paragraph to search for. Provide only the number of the Paragraph, without any letters like 'bis' or 'ter'. If not specified, all paragraphs of the article are returned as a list.")
    lawbook: str = Field(..., description="The lawbook to search for. Currently supported is OR and ZGB. This parameter must be supplied.")
 

@tool(args_schema=ParagraphFinder)
def get_paragraphs(lawbook: str, articleNum: int, articleNumMinor: str = None, paragraphNum: int = None) -> dict:
    """Fetch all paragraph for selected lawbook, article number and paragraph. if an optional parameter is not supplied, all paragraphs of the article are returned."""
    
    BASE_URL = "http://127.0.0.1:8000/lawbook/getParagraphs"
    
    # Parameters for the request
    params = {
        'bookName': lawbook,
        'articleNum': articleNum,
        'articleNumMinor': articleNumMinor,
        'paragraphNum' : paragraphNum,
    }

    # Make the request
    print(f"Making request to {BASE_URL} with params: {params}")
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    return results
    

In [9]:
functions = [
    format_tool_to_openai_function(f) for f in [
        get_paragraphs
    ]
]
model = ChatOpenAI(temperature=0, model=lm_name).bind(functions=functions)

In [22]:
model.invoke("what is the exact paragraph of Art. 12 OR?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_paragraphs', 'arguments': '{\n  "articleNum": 12,\n  "lawbook": "OR"\n}'}})

In [10]:
# define the prompt
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful and smart legal assistant to a Swiss layer"),
    ("user", "{input}"),
])


In [11]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "get_paragraphs": get_paragraphs, 
        }
        return tools[result.tool].run(result.tool_input)

In [12]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [21]:
result = chain.invoke({"input": "what is the exact paragraph of Art. 1 OR?"})
print("Result: ", result)

Making request to http://127.0.0.1:8000/lawbook/getParagraphs with params: {'bookName': 'OR', 'articleNum': 1, 'articleNumMinor': None, 'paragraphNum': None}
Result:  [{'id': '4451', 'bookName': 'OR', 'levelId': 'part_1/tit_1/chap_1/lvl_A', 'paragraphEid': 'art_1/para_1', 'paragraphText': 'Zum Abschlusse eines Vertrages ist die übereinstimmende gegenseitige Willensäusserung der Parteien erforderlich.', 'footnoteText': None, 'articleNum': 1, 'articleNumMinor': None, 'paragraphNum': 1, 'paragraphNumMinor': None}, {'id': '4452', 'bookName': 'OR', 'levelId': 'part_1/tit_1/chap_1/lvl_A', 'paragraphEid': 'art_1/para_2', 'paragraphText': 'Sie kann eine ausdrückliche oder stillschweigende sein.', 'footnoteText': None, 'articleNum': 1, 'articleNumMinor': None, 'paragraphNum': 2, 'paragraphNumMinor': None}]


### Now let's parse out the results and give it back

In [29]:
from langchain.output_parsers import StructuredOutputParser
import pandas as df


def parse_law_data(json_data)->list:
    parsed_data = []
    for entry in json_data:
        row = {
            'Lawbook': entry['bookName'],
            'Article Number': entry['articleNum'],
            'Paragraph Number': entry.get('paragraphNum', ''),
            'Paragraph Text': entry['paragraphText']
        }
        parsed_data.append(row)
    return parsed_data


table_data = parse_law_data(result)
data = df.DataFrame(table_data)
#print all data in a table 'data' in a pretty format
data


,Lawbook,Article Number,Paragraph Number,Paragraph Text
0,OR,1,1,Zum Abschlusse eines Vertrages ist die überein...
1,OR,1,2,Sie kann eine ausdrückliche oder stillschweige...
